## Reddit Scraping
Documentation reference: https://praw.readthedocs.io/en/stable/

In [40]:
import requests
import pandas as pd
import praw
from praw.models import MoreComments

import os
import zipfile

os.makedirs("../output", exist_ok=True)
from dotenv import load_dotenv
load_dotenv()


True

# Scrape post from related subreddits
Usage: Consumer sentiment analysis 

In [ ]:

reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
)

# "Skincare_Addiction", "asianskincare", "Blackskincare","SkincareAddicts"
subreddit_list = ["SkincareAddiction",]

all_posts = []

# get the top 20 post from each subreddit (don't know rate limit so 20 for now)
# Documentation: https://praw.readthedocs.io/en/stable/code_overview/models/subreddit.html
keywords = [
    "CeraVe", "La Roche-Posay", "The Ordinary",
    "Niacinamide", "Retinol", "Vitamin C serum",
    "Paula's Choice", "COSRX", "Hydrocolloid", "Sunscreen"
]
def get_top_comments(post):

    post.comments.replace_more(limit=0)  

    top_comments = []
    for comment in post.comments:
        if isinstance(comment, MoreComments):
            continue 
        top_comments.append(comment.body)
        if len(top_comments) == 5:
            break
    return top_comments
        
for sub in subreddit_list:
    try:
        post_collection = reddit.subreddit(sub).top(limit=20)  
        for post in post_collection: 
            top_comments = get_top_comments(post)
            all_posts.append({
            "subreddit_name": sub,
            "post_id": post.id,
            "title": post.title,
            "description": post.selftext,
            "score": post.score,
            "num_comments": post.num_comments,
            "top_comments": top_comments,
            "upvote_ratio": post.upvote_ratio,
           })
    except Exception as e:
        print(f"Error scraping {sub}: {e}")
        continue
    
subreddit_df = pd.DataFrame(all_posts)
subreddit_df
subreddit_df.to_csv("../output/subreddit_data.csv", index=False)



In [47]:
skincare_df = pd.read_csv("../output/subreddit_data.csv")
skincare_df.head()


,subreddit_name,post_id,title,description,score,num_comments,top_comments,upvote_ratio
0,SkincareAddiction,8pv3gg,[B&A] Finished my 5 month course of accutane j...,NaN,22422,276,['IM JUST SO FUCKING HAPPY FOR YOU. OH MY GOD ...,1.00
1,SkincareAddiction,frxxqw,[misc]A few days ago I mentioned being a nurse...,NaN,21044,250,"['That’s so sweet, you deserve it and so much ...",0.99
2,SkincareAddiction,iahv1s,[PSA] Probably your daily reminder by now ♡,NaN,18530,99,"[""Developing a skincare routine got me through...",0.99
3,SkincareAddiction,i1gl2x,[Humor] Too cruel?,NaN,18514,312,['@ Mean Girls when Cady gives Regina foot cre...,0.99
4,SkincareAddiction,gg0j06,[selfie] I have been consistently going to wor...,NaN,18229,1229,"['Hello everyone! Sorry for the delay, the com...",0.99
